## BÀI THỰC HÀNH SỐ 4: THỰC HÀNH KỸ THUẬT NAIVE BAYES

Thuật toán Naive Bayes hoạt động dựa trên Định lý Bayes và giả định rằng các đặc trưng độc lập với nhau
$$
P(c|X) = \frac{ P(X|c) * P(c) }{P(X)}
$$
Trong đó:
$P(c|X)$ là xác suất hậu nghiệm của lớp $y$ khi biết các đặt trưng $X$
$P(X∣c)$ là xác suất có điều kiện của các đặc trưng $X$ khi biết lớp $c$
$P(c)$ là xác suất tiên nghiệm của lớp $c$
$P(X)$ là xác suất của các đặc trưng $X$.

Bước 1: Khởi tạo các thuộc tính cho từng thực thể của class MultinomialNaiveBayes



Bước 2: Tính toán xác suất tiên nghiệm của các lớp $y$:
$P(y)$ được tính bằng tỉ lệ số điểm dữ liệu trong tập huấn luyện rơi vào class này chia cho tổng số lượng dữ liệu trong tập huấn luyện

$$
P(Y_i) = \frac{N_i}{N}
$$

Trong đó: 
$P(Y_i)$: xác suất tiên nghiệm của lớp $Y_i$
$N_i$: số lượng mẫu thuộc lớp $Y_i$
$N$: tổng số lượng mẫu trong toàn bộ tập dữ liệu


Bước 3: Tính $p(x_i|y)$ (likelihood) là tỷ lệ với tần suất từ thứ i (hay đặc trưng thứ i trong trường hợp tổng quát) xuất hiện trong các văn bản của class y. 
$$
p(x_i| y) = log(\frac{N_{yi} + \alpha}{N_y + d \alpha})
$$
Trong đó:
$N_{yi}$ là tổng số lần từ thứ $i$ xuất hiện trong các văn bản của class $y$, nó được tính bằng tổng của tất cả các thành phần thứ i của các vector đặc trưng ứng với lớp $y$.
$N_y$ là tổng số từ (kể cả lặp) xuất hiện trong class $y$. Nói các khác, nó bằng tổng độ dài của toàn bộ các văn bản thuộc vào class $y$. 
$\alpha$ là hệ số Laplace smoothing

In [1]:
import numpy as np

class MultinomialNaiveBayes:
    def __init__(self, alpha=1.0):
        self.alpha = alpha  # Tham số làm mịn Laplace
        self.class_prior = None  # Xác suất tiên nghiệm của các lớp
        self.feature_prob = None  # Xác suất có điều kiện của các đặc trưng cho lớp
        self.classes = None  # Các lớp duy nhất

    def fit(self, X, y):
        """Huấn luyện mô hình với dữ liệu."""
        self.classes, self.class_prior = self.compute_class_prior(y)
        self.feature_prob = self.compute_feature_prob(X, y)

    def compute_class_prior(self, y):
        """Tính xác suất tiên nghiệm của từng lớp."""
        n_samples = len(y)
        classes, counts = np.unique(y, return_counts=True)
        class_prior = counts / n_samples
        return classes, class_prior
    

    def compute_feature_prob(self, X, y):
        """Tính xác suất có điều kiện của từng đặc trưng cho từng lớp."""
        n_features = X.shape[1]
        n_classes = len(self.classes)
        feature_prob = np.zeros((n_classes, n_features))
        
        for i, c in enumerate(self.classes):
            X_c = X[y == c]
            feature_prob[i, :] = (np.sum(X_c, axis=0) + self.alpha) / (np.sum(X_c) + self.alpha * n_features)
        
        return feature_prob

    def compute_posterior(self, X):
        """Tính xác suất hậu nghiệm cho mỗi mẫu."""
        n_samples = X.shape[0]
        n_classes = len(self.classes)
        posterior = np.zeros((n_samples, n_classes))
        
        for i in range(n_samples):
            for j in range(n_classes):
                posterior[i, j] = np.log(self.class_prior[j]) + np.sum(np.log(self.feature_prob[j, :]) * X[i])
        
        return posterior

    def predict(self, X):
        """Dự đoán nhãn lớp cho dữ liệu đầu vào."""
        posterior = self.compute_posterior(X)
        return self.classes[np.argmax(posterior, axis=1)]
    
    def compute_accuracy(self, y_pred, y_true):
        correct_predict = np.sum(y_true == y_pred)
        total_predict = len(y_true)
        accuracy = correct_predict / total_predict

        return accuracy



In [2]:
#Ví dụ sử dụng

X_train = np.array([
    [2,1,0,0,0,0],
    [2,0,1,0,0,0],
    [1,0,0,1,0,0],
    [1,0,0,0,1,1]
])
y_train = np.array([1,1,1,0])

X_test = np.array([[3,0,0,0,1,1]])
y_test = np.array([[1]])

#Tạo mô hình phân loại Multinomial Naive Bayes
clf = MultinomialNaiveBayes()

#Huấn luyện mô hình
clf.fit(X_train, y_train)

# Dự đoán dữ liệu kiểm thử
y_pred = clf.predict(X_test)

# Tính toán độ chính xác 
accuracy = clf.compute_accuracy(y_pred, y_test)
print(f"Accuracy: {accuracy * 100:.2f}%")
print("Predicted classes:", y_pred)

Accuracy: 100.00%
Predicted classes: [1]
